In [1]:
import torch
import torch.nn as nn

### 定义

给定输入向量 **x**，其形状为 $n\_features$，对于批量大小为 $batch\_size$，输入矩阵为 $\mathbf{X} \in \mathbb{R}^{batch\_size \times n\_features}$。

模型的参数包括：

- $\mathbf{w}_0 \in \mathbb{R}$：偏置项
- $\mathbf{w}_1 \in \mathbb{R}^{n\_features \times 1}$：线性权重
- $\mathbf{W}_2 \in \mathbb{R}^{n\_features \times n\_features}$：二次交叉权重

### 模型公式

1. **线性部分**：

$$
   \text{linear\_output} = \mathbf{w}_0 + \mathbf{X} \mathbf{w}_1
$$

   其中，$\mathbf{X} \mathbf{w}_1$ 表示输入向量与线性权重的矩阵乘法。

2. **二次交叉部分**：

$$
   \text{cross\_output} = \sum_{i=1}^{n\_features} \sum_{j=1}^{n\_features} W_{2,ij} \cdot X_i \cdot X_j
$$

   这部分通过 Hadamard（逐元素）乘积实现，用批量矩阵乘法表示为：
$$
   \text{cross\_output} = \sum (\mathbf{W}_2 \odot (\mathbf{X}^T \mathbf{X}))
$$
   这里 $\mathbf{X}^T \mathbf{X}$ 是对每个输入样本的外积，$\odot$ 表示逐元素乘积，$\sum$ 表示对所有元素求和。

3. **最终输出**：

$$
   \text{logits} = \sigma(\text{linear\_output} + \text{cross\_output})
$$

   其中，$\sigma(z) = \frac{1}{1 + e^{-z}}$ 是 Sigmoid 激活函数。

### 总公式

将上述部分结合，模型可以表示为：
$$
\text{logits} = \sigma\left(\mathbf{w}_0 + \mathbf{X} \mathbf{w}_1 + \sum_{i=1}^{n\_features} \sum_{j=1}^{n\_features} W_{2,ij} \cdot X_i \cdot X_j\right)
$$
这个公式表示模型对输入向量 $\mathbf{X}$ 进行线性加二次交叉的特征组合，然后通过 Sigmoid 激活函数计算概率输出。

线性加二次交叉的特征组合，然后通过 Sigmoid 激活函数计算概率输出。

In [ ]:
class POLY2(nn.Module):
    def __init__(self, n_features):
        """
        初始化 POLY2 类，定义模型的参数。
        
        参数:
        - n_features (int): 输入特征的数量。
        """
        
        super(POLY2, self).__init__()
        self.w0 = nn.init.xavier_uniform_(nn.Parameter(torch.empty(1, 1))) # w0 是一个标量，用于偏置项
        self.w1 = nn.init.xavier_uniform_(nn.Parameter(torch.empty(n_features, 1))) # w1 是一个形状为 [n_features, 1] 的矩阵，用于线性部分的权重
        self.w2 = nn.init.xavier_uniform_(nn.Parameter(torch.empty(n_features, n_features))) # w2 是一个形状为 [n_features, n_features] 的矩阵，用于二次交叉项

    def crossLayer(self, x):
        """
        计算二次交叉项。
        
        参数:
        - x (Tensor): 输入张量，形状为 [batch_size, n_features]。
        
        返回:
        - cross_out (Tensor): 二次交叉项的输出，形状为 [batch_size, 1]。
        """
        x = x.unsqueeze(2) # 将输入张量扩展为 [batch_size, n_features, 1]
        x_transpose = x.transpose(1, 2) # 计算输入的转置，形状为 [batch_size, 1, n_features]
        x_cross = torch.bmm(x_transpose, x) # 计算每个样本的二次交叉矩阵乘法，结果形状为 [batch_size, n_features, n_features]
        cross_out = torch.sum(self.w2 * x_cross, dim=(1, 2)).reshape(-1, 1) # 对交叉结果和 w2 做 Hadamard 乘积，并求和
        
        return cross_out

    def forward(self, x):
        """
        前向传播逻辑。
        
        参数:
        - x (Tensor): 输入张量，形状为 [batch_size, n_features]。
        
        返回:
        - logits (Tensor): 模型的输出概率，形状为 [batch_size, 1]。
        """
        lr_out = self.w0 + torch.matmul(x, self.w1) # 计算线性模型的输出部分 lr_out = w0 + x * w1
        cross_out = self.crossLayer(x) # 计算二次交叉项的输出
        logits = torch.sigmoid(lr_out + cross_out) # 计算最终的输出，并应用 sigmoid 激活函数
        
        return logits